In [5]:
import os
os.environ["OPENAI_API_KEY"] = "OPENAI API KEY"
from sqlalchemy import create_engine, text, select
from sqlalchemy import create_engine
from langchain_core.messages import AIMessage, SystemMessage
import requests
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from databases import Database
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

In [ ]:
'''
To create sample data in the form of a sql database, I will be using sqlite to create the database.
'''

In [1]:
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('bank_statement.db')
cursor = conn.cursor()

# Create table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS bank_statement (
        date TEXT,
        type TEXT,
        description TEXT,
        paid_in REAL,
        paid_out REAL,
        balance REAL
    )
''')

# Data to be inserted
data = [
    ("22 Oct 2014", "AUTOMATED PAY IN", "650274051211-CHB", 190.40, None, 1803.9),
    ("22 Oct 2014", "DIGITAL BANKING", "CALL REF NO. 3442, FROM A/C 2222222", None, 140.00, 1613.5),
    ("24 Oct 2014", "Faster Payment", "Amazon", None, 132.30, 1475.3),
    ("24 Oct 2014", "BACS", "Tebay Trading Co.", 515.22, None, 1341.2),
    ("25 Oct 2014", "Faster Payment", "Morrisons Petrol", None, 80.00, 928.96),
    ("26 Oct 2014", "BACS", "Business Loan", 20000.00, None, 745.96),
    ("26 Oct 2014", "BACS", "James White Media", 2461.55, None, 20745.98),
    ("27 Oct 2014", "Faster Payment", "ATM High Street", 100.00, None, 18584.43),
    ("01 Nov 2014", "BACS", "Acorn Advertising Studies", 150.00, None, 18184.43),
    ("01 Nov 2014", "BACS", "Marriott Hotel", 177.00, None, 18034.43),
    ("01 Nov 2014", "Faster Payment", "Abellio Scotrail Ltd", 122.22, None, 17857.43),
    ("01 Nov 2014", "CHQ", "Cheque 0000234", 1200.00, None, 17735.21),
    ("01 Dec 2014", "Int. Bank", "Interest Paid", 9.33, None, 17613.21),
    ("01 Dec 2014", "DD", "OVO Energy", None, 2470.00, 16564.54),
    ("21 Dec 2014", "BACS", "Various Payment", None, 10526.40, 14074.54),
    ("21 Dec 2014", "BACS", "HMRC", None, 1000.00, 13048.14),
    ("21 Dec 2014", "DD", "DVLA", None, 280.00, 2548.14)
]

# Insert data into table
cursor.executemany('''
    INSERT INTO bank_statement (date, type, description, paid_in, paid_out, balance)
    VALUES (?, ?, ?, ?, ?, ?)
''', data)

# Commit changes and close connection
conn.commit()
conn.close()

print("Data inserted successfully!")


Data inserted successfully!


In [6]:
db = SQLDatabase.from_uri("sqlite:///bank_statement.db")
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0, streaming=True, verbose=False)
top_k = 10
toolkit = SQLDatabaseToolkit(llm=llm, db=db)
dialect = toolkit.dialect
prefix = f"""You are an agent designed to interact with a SQL database to answer questions.

DO NOT use 'multi_tool_use.parallel' tool. multi_tool_use.parallel is not a valid tool, try one of [sql_db_query, sql_db_schema, sql_db_list_tables, sql_db_query_checker].

Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (CREATE, INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.
"""

suffix = f"""
I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
"""

messages = [
    SystemMessage(content=prefix),
    HumanMessagePromptTemplate.from_template("{input}"),
    AIMessage(content=suffix),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
]
prompt = ChatPromptTemplate.from_messages(messages)

agent_executor = create_sql_agent(
    llm, db=db, agent_type="openai-tools", verbose=False, prompt=prompt)

In [7]:
query = "How much did I spend in total?"
response = agent_executor.invoke(query)['output']

In [8]:
response

'You have spent a total of $14,628.70.'